In [66]:
import numpy as np
import pandas as pd
import keras
from keras import layers, models
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, Adamax, SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from google.colab import files
import math

# Esimerkki
'''

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

'''

# Ladataan data CSV-tiedostosta
file_path = "data_from_mysql_where_g16.csv"
data = pd.read_csv(file_path)

# Tarkastetaan datan rakenne
print(data.head())

# Suodatetaan pois kaikki rivit, joissa 'sensorvalue_d' on 0
data_filtered = data[data['sensorvalue_d'] != 0]

x_data = data_filtered[['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c']].values  # x, y, z
y_data = data_filtered['sensorvalue_d'].values # suunta

# Skaalaus [0, 1] väliin
def scale_to_unit_interval(data):
    min_vals = np.min(data, axis=0)  # Minimiarvot sarakkeittain
    max_vals = np.max(data, axis=0)  # Maksimiarvot sarakkeittain
    # Varmistetaan, ettei ole nollalla jakamista
    ranges = max_vals - min_vals
    ranges[ranges == 0] = 1  # Jos max == min, käytetään 1:stä

    return (data - min_vals) / ranges  # Skaalaus [0, 1] väliin

# Skaalataan syötteet
x_data_scaled = scale_to_unit_interval(x_data)

# Skaalataan syötteet [0, 1] väliin käyttäen StandardScaler
#scaler = StandardScaler()
#x_data = scaler.fit_transform(x_data.astype('float32'))

# Suuntaa on 6 luokkaa
num_classes = 6
y_data = keras.utils.to_categorical(y_data - 1, num_classes)

x_train, x_test, y_train, y_test = train_test_split(x_data_scaled, y_data, test_size=0.2, random_state=42)

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")

# Määritellään malli
model = keras.Sequential(
    [
        keras.Input(shape=(x_train.shape[1],)),  # Syötemuoto (x, y, z)
        layers.Dense(256, activation="relu"),  # Ensimmäinen tiheä kerros
        layers.Dropout(0.2),  # Dropout, jotta ylikoulutus ei tapahdu
        layers.Dense(128, activation="relu"),  # Toinen tiheä kerros
        layers.LeakyReLU(negative_slope=0.2),  # Vaihtoehto ReLU:lle
        layers.Dense(64, activation="relu"),  # Kolmas tiheä kerros
        layers.Dense(32, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),  # Lopullinen luokittelukerros
    ]
)

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Koulutetaan mallia
batch_size = 128
epochs = 100

optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping])

# Arvioidaan malli testidatalla
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Tallennetaan malli (painot ja rakenne)
model.save('my_model.keras')

# Osa 2

# Haetaan mallin painot
weights = model.get_weights()

# Painot ovat lista, jossa on numpy-taulukoita
for idx, weight in enumerate(weights):
    print(f"Painojen ja biasin {idx} muoto: {weight.shape}")
    #print(f"Painot ja bias {idx}:", weight)

# Aktivointifunktiot
def relu(x):
    return np.maximum(0, x)  # ReLU-funktio

'''
def softmax(x):
    exp_values = np.exp(x - np.max(x))  # Stabiili softmax
    return exp_values / exp_values.sum(axis=-1, keepdims=True)
'''
'''
# Softmax ilman NumPytä
def softmax(x):
    max_val = max(x)  # Löydetään suurin arvo numeron stabiliteetin vuoksi
    exp_values = [math.exp(i - max_val) for i in x]  # Lasketaan eksponentit ilman NumPytä
    total = sum(exp_values)  # Lasketaan summa ilman NumPytä
    return [i / total for i in exp_values]  # Normalisoidaan ilman NumPytä
'''

# Aktivointifunktio Softmax
def softmax(z):
    exp_values = [2.718 ** i for i in z]  # Eksponenttiarvot
    total = sum(exp_values)  # Lasketaan summan eksponentit
    return [exp_value / total for exp_value in exp_values]  # Jaa eksponenttiarvot kokonaisuudella

# Syöte
for i in range(len(data_filtered)):
    # Haetaan kunkin rivin arvot
    x = data_filtered['sensorvalue_a'].values[i]
    y = data_filtered['sensorvalue_b'].values[i]
    z = data_filtered['sensorvalue_c'].values[i]

    # Muutetaan syöte oikeaan muotoon
    input_data = np.array([x, y, z]).reshape(1, -1)  # Muutetaan muotoon (1, 3)

#x = data_filtered['sensorvalue_a'].values[0]  # Ensimmäinen arvo sarakkeesta x
#y = data_filtered['sensorvalue_b'].values[0]  # Ensimmäinen arvo sarakkeesta y
#z = data_filtered['sensorvalue_c'].values[0]  # Ensimmäinen arvo sarakkeesta z

#input_data = np.array([x, y, z]).reshape(1, -1)  # Muutetaan muotoon (1, 3)

print("Input data shape:", input_data.shape)

# Skaalaus [0, 1] väliin
def scale(data):
    min_vals = np.min(data, axis=0)  # Minimiarvot sarakkeittain
    max_vals = np.max(data, axis=0)  # Maksimiarvot sarakkeittain
     # Varmistetaan, ettei ole nollalla jakamista
    ranges = max_vals - min_vals
    ranges[ranges == 0] = 1  # Jos max == min, käytetään 1:stä

    return (data - min_vals) / ranges  # Skaalaus [0, 1] väliin

# Skaalataan syöte
input_data_scaled = scale(input_data.astype('float32'))

# Varmistetaan, että data on skaalattu oikein
#scaler = StandardScaler()
#input_data_scaled = scaler.fit_transform(input_data.astype('float32'))

print("Input data scaled shape:", input_data_scaled.shape)

# Oikeat painot ja biasit
weights_0, bias_0 = weights[0], weights[1]
weights_1, bias_1 = weights[2], weights[3]
weights_2, bias_2 = weights[4], weights[5]
weights_3, bias_3 = weights[6], weights[7]
weights_4, bias_4 = weights[8], weights[9]

'''
# Etenee syötteestä piilokerrosten kautta ulostuloon (numpy)
def forward_propagation(input_data):
    # Piilokerros 1
    z0 = np.dot(input_data, weights_0) + bias_0
    a0 = relu(z0)  # Aktivointi

    # Piilokerros 2
    z1 = np.dot(a0, weights_1) + bias_1
    a1 = relu(z1)  # Aktivointi

    # Piilokerros 3
    z2 = np.dot(a1, weights_2) + bias_2
    a2 = relu(z2)  # Aktivointi

    # Piilokerros 4
    z3 = np.dot(a2, weights_3) + bias_3
    a3 = relu(z3)  # Aktivointi

    # Ulostulokerros
    z4 = np.dot(a3, weights_4) + bias_4
    output = softmax(z4)  # Softmax aktivointi

    return output
'''
'''
# Etenee syötteestä piilokerrosten kautta ulostuloon (yhdellä arvolla)
def forward_propagation(input_data):
    # Piilokerros 1
    z0 = [sum(i * j for i, j in zip(input_data[0], weights_0[:, k])) + bias_0[k] for k in range(weights_0.shape[1])]
    a0 = [relu(z) for z in z0]  # Aktivointi

    # Piilokerros 2
    z1 = [sum(i * j for i, j in zip(a0, weights_1[:, k])) + bias_1[k] for k in range(weights_1.shape[1])]
    a1 = [relu(z) for z in z1]  # Aktivointi

    # Piilokerros 3
    z2 = [sum(i * j for i, j in zip(a1, weights_2[:, k])) + bias_2[k] for k in range(weights_2.shape[1])]
    a2 = [relu(z) for z in z2]  # Aktivointi

    # Piilokerros 4
    z3 = [sum(i * j for i, j in zip(a2, weights_3[:, k])) + bias_3[k] for k in range(weights_3.shape[1])]
    a3 = [relu(z) for z in z3]  # Aktivointi

    # Ulostulokerros
    z4 = [sum(i * j for i, j in zip(a3, weights_4[:, k])) + bias_4[k] for k in range(weights_4.shape[1])]
    output = softmax(z4)  # Softmax aktivointi

    return output
'''

#Etenee syötteestä piilokerrosten kautta ulostuloon
def forward_propagation(input_data):
    # Piilokerros 1
    z0 = []
    for k in range(len(weights_0[0])):  # Käydään läpi piilokerroksen neuronit
        z0_value = sum(input_data_scaled[i] * weights_0[i][k] for i in range(len(input_data_scaled))) + bias_0[k]
        z0.append(z0_value)
    a0 = [relu(z) for z in z0]  # Aktivointi

    # Piilokerros 2
    z1 = []
    for k in range(len(weights_1[0])):
        z1_value = sum(a0[i] * weights_1[i][k] for i in range(len(a0))) + bias_1[k]
        z1.append(z1_value)
    a1 = [relu(z) for z in z1]  # Aktivointi

    # Piilokerros 3
    z2 = []
    for k in range(len(weights_2[0])):
        z2_value = sum(a1[i] * weights_2[i][k] for i in range(len(a1))) + bias_2[k]
        z2.append(z2_value)
    a2 = [relu(z) for z in z2]  # Aktivointi

    # Piilokerros 4
    z3 = []
    for k in range(len(weights_3[0])):
        z3_value = sum(a2[i] * weights_3[i][k] for i in range(len(a2))) + bias_3[k]
        z3.append(z3_value)
    a3 = [relu(z) for z in z3]  # Aktivointi

    # Ulostulokerros
    z4 = []
    for k in range(len(weights_4[0])):
        z4_value = sum(a3[i] * weights_4[i][k] for i in range(len(a3))) + bias_4[k]
        z4.append(z4_value)
    output = softmax(z4)  # Softmax aktivointi

    # Muotoillaan tulos muotoon (1, 6) ja pyöristetään arvoja samalla tarkkuudella kuin model.predict
    output = np.array(output)  # Varmistetaan, että output on numpy-taulukko
    output = np.round(output, 6).reshape(1, -1)  # Muotoillaan ja pyöristetään

    # Tulostetaan arvot, jotka ovat indekseissä 0, 3, 6, 9, jne.
    selected_values = output[0][::3]  # Valitaan arvot, jotka ovat paikoissa 0, 3, 6, 9 jne.

    return selected_values

    #return output

# Asetetaan NumPy:n tulostustapa niin, että ei käytetä tieteellistä merkintää
np.set_printoptions(precision=6, suppress=True)

# Lasketaan tulos syötteelle (x, y, z)
result = forward_propagation(input_data_scaled)
print("\nVerkon ulostulo: (forward_propagation):", result)

# Lasketaan ennuste koulutetulla mallilla
prediction = model.predict(input_data_scaled)
print("\nEnnuste (model.predict):", prediction)

# Lasketaan ero tulosten välillä
# Ensimmäinen vaihe on varmistaa, että molemmat result ja prediction ovat numpy-taulukkoja
result = np.array(result).reshape(-1)  # Muutetaan result tasaiseksi taulukoksi
prediction = np.array(prediction).reshape(-1)  # Muutetaan prediction tasaiseksi taulukoksi

# Lasketaan ero
difference = np.abs(result - prediction)  # Lasketaan itseisarvoero

# Tulostetaan ero
print("\nEro (absoluuttinen ero result ja prediction välillä):", difference)

# Keskimääräinen ero
mean_difference = np.mean(difference)
print("\nKeskimääräinen ero: ", mean_difference)

# Tallennetaan painot ja biasit header-tiedostoon
header_file = "neuroverkonKertoimet.h"

with open(header_file, "w") as f:
    f.write("#ifndef NEUROVERKONKERTOIMET_H\n")
    f.write("#define NEUROVERKONKERTOIMET_H\n\n")

    # Kirjoitetaan painot ja biasit jokaiselle kerrokselle
    for idx, weight in enumerate(weights):
        if len(weight.shape) == 2:  # Painot (matriisi)
            f.write(f"float weights_{idx}[{weight.shape[0]}][{weight.shape[1]}] = {{\n")
            for row in weight:
                f.write("    {" + ", ".join(map(str, row)) + "},\n")
            f.write("};\n\n")
        elif len(weight.shape) == 1:  # Bias (vektori)
            f.write(f"float biases_{idx}[{weight.shape[0]}] = {{")
            f.write(", ".join(map(str, weight)))
            f.write("};\n\n")

    f.write("#endif // NEUROVERKONKERTOIMET_H\n")

   id            timestamp  groupid  from_mac  to_mac  sensorvalue_a  \
0  39  2024-11-14 12:30:54       16         0       0         1510.0   
1  40  2024-11-14 12:30:54       16         0       0         1503.0   
2  41  2024-11-14 12:30:54       16         0       0         1511.0   
3  42  2024-11-14 12:30:55       16         0       0         1509.0   
4  43  2024-11-14 12:30:56       16         0       0         1505.0   

   sensorvalue_b  sensorvalue_c  sensorvalue_d  sensorvalue_e  sensorvalue_f  
0         1405.0         1812.0            0.0            0.0              0  
1         1413.0         1818.0            0.0            0.0              0  
2         1416.0         1814.0            0.0            0.0              0  
3         1412.0         1814.0            0.0            0.0              0  
4         1410.0         1804.0            0.0            0.0              0  
x_train shape: (2210, 3)
x_test shape: (553, 3)


Model: "sequential_64"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_320 (Dense)                    │ (None, 256)                 │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_64 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_321 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_64 (LeakyReLU)           │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_322 (Dense)                    │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_323 (Dense)                    │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_324 (Dense)                    │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,454 (173.65 KB)

 Trainable params: 44,454 (173.65 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 127ms/step - accuracy: 0.2251 - loss: 1.7627 - val_accuracy: 0.5249 - val_loss: 1.6338
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4789 - loss: 1.6095 - val_accuracy: 0.5249 - val_loss: 1.3612
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5339 - loss: 1.2982 - val_accuracy: 0.6199 - val_loss: 1.0630
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6170 - loss: 1.0528 - val_accuracy: 0.6109 - val_loss: 0.9119
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6526 - loss: 0.9154 - val_accuracy: 0.6199 - val_loss: 0.8436
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6356 - loss: 0.8706 - val_accuracy: 0.6742 - val_loss: 0.8183
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6389 - loss: 0.8613 - val_accuracy: 0.6606 - val_loss: 0.7976
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6468 - loss: 0.8263 - val_accuracy: 0.6923